In [1]:
from pyspark import SparkContext, SparkConf, HiveContext
from pyspark.sql import SparkSession, Row, functions, Column
from pyspark.sql.types import *
import pyspark.sql.functions as F

In [2]:
def create_config(app_name,
                  memory=1,
                  instances=1,
                  cores=2,
                  max_result_size=50,
                  driver_memory=1):
    """
    simple wrapper for SparkConfig().set()
    :param memory: int, max provided memory, in gigabytes
    :param instances:
    :param cores:
    :param max_result_size: int, max result size in gigabytes
    :return: pyspark.conf.SparkConf object
    """
    spark_conf = SparkConf().setAppName(app_name)
    spark_conf.set('spark.executor.memory', f'{memory}g')
#     spark_conf.set('spark.executor.instances', f'{instances}')
    spark_conf.set('spark.executor.cores', f'{cores}')
#     spark_conf.set('spark.driver.maxResultSize', f'{max_result_size}g')
    spark_conf.set('spark.driver.memory', f'{driver_memory}g')
#     spark_conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")
#     spark_conf.set('hive.exec.dynamic.partition.mode','nonstrict')
    return spark_conf

def create_session(conf,):
    return SparkSession.Builder().config(conf=conf).getOrCreate()

ses = create_session(create_config('tmp_writer'))

In [3]:
df = ses.read.options(delimiter=';').csv('admin_lev8.txt')

In [4]:
df.show()

+--------------------+-------+-------+---------+--------+-------+---------+--------+---------+--------+---------+--------------------+---------+-----------------+---------+--------------------+---------+--------------------+---------+--------+---------+--------+---------+--------+----------+---------+
|                 _c0|    _c1|    _c2|      _c3|     _c4|    _c5|      _c6|     _c7|      _c8|     _c9|     _c10|                _c11|     _c12|             _c13|     _c14|                _c15|     _c16|                _c17|     _c18|    _c19|     _c20|    _c21|     _c22|    _c23|      _c24|     _c25|
+--------------------+-------+-------+---------+--------+-------+---------+--------+---------+--------+---------+--------------------+---------+-----------------+---------+--------------------+---------+--------------------+---------+--------+---------+--------+---------+--------+----------+---------+
|                NAME|NAME_EN|NAME_RU|ADMIN_LVL|OSM_TYPE| OSM_ID|ADMIN_L1D|ADMIN_L1|ADMIN_L